In [1]:
from pre_process import pre_process_data

c:\Users\91790\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import pandas as pd 
data = pre_process_data(pd.read_csv('hotel_bookings.csv'))

In [3]:
df = data.copy()

In [4]:
from feature_engineeing import build_features_for_rag, build_features_for_analytics

In [5]:
data = build_features_for_rag(df)

In [10]:
rag_df, gm = build_features_for_rag(df, verbose=True)

Processing Revenue Trends...
Processing Booking and Cancellations Trends...
Processing Occupancy and Demand Trends...
Processing Customer and Demographic Features...
Processing Time Based Trends...


### Feature Engineering for RAG

Revenue trends

In [11]:
# Revenue per booking 
df['total_revenue'] = df['adr'] * (df['adults'] + df['children'] + df['babies'])

# Global metrics 
average_revenue_per_booking = df['total_revenue'].mean()  # Single value
revenue_per_month = df.groupby(['year', 'month'])['total_revenue'].sum().reset_index()  # Aggregated
revenue_per_market_segment = df.groupby('market_segment')['total_revenue'].sum().reset_index()
revenue_per_meal_plan = df.groupby('meal')['total_revenue'].sum().reset_index()


In [12]:
revenue_per_month.head(6)

,year,month,total_revenue
0,2015,7,588400.72
1,2015,8,898091.10
2,2015,9,917673.37
3,2015,10,716712.14
4,2015,11,236721.01
5,2015,12,428656.27


In [13]:
revenue_per_market_segment

,market_segment,total_revenue
0,Aviation,24073.68
1,Complementary,3785.29
2,Corporate,476668.85
3,Direct,3250387.29
4,Groups,2848790.96
5,Offline TA/TO,3987887.18
6,Online TA,14941809.06
7,Undefined,78.00


In [14]:
revenue_per_meal_plan

,meal,total_revenue
0,BB,19256210.43
1,FB,199229.25
2,HB,3876016.85
3,SC,1979224.34
4,Undefined,222799.44


Booking and Cancellation Features

In [15]:
# Total cancellations per hotel
cancellations_by_hotel = df.groupby('hotel')['is_canceled'].sum().reset_index()
# Total cancellations per country
cancellations_by_country = df.groupby('country')['is_canceled'].sum().reset_index()

# Total cancellations per customer type
cancellations_by_customer_type = df.groupby('customer_type')['is_canceled'].sum().reset_index()

# Total cancellations per season (holiday vs. non-holiday)
cancellations_by_season = df.groupby('is_holiday_season')['is_canceled'].sum().reset_index()

# Overall cancellation rate
overall_cancellation_rate = df['is_canceled'].mean() * 100  

In [16]:
cancellations_by_hotel

,hotel,is_canceled
0,City Hotel,33102
1,Resort Hotel,11122


In [17]:
cancellations_by_country

,country,is_canceled
0,ABW,0
1,AGO,205
2,AIA,0
3,ALB,2
4,AND,5
...,...,...
172,VGB,1
173,VNM,2
174,ZAF,31
175,ZMB,1


In [18]:
cancellations_by_customer_type

,customer_type,is_canceled
0,Contract,1262
1,Group,59
2,Transient,36514
3,Transient-Party,6389


In [19]:
cancellations_by_season

,is_holiday_season,is_canceled
0,False,30065
1,True,14159


In [20]:
overall_cancellation_rate

37.041628277075134

Occupancy and Demand Features

In [21]:
import numpy as np

In [22]:
# Total stay duration per booking
df['average_stay_duration'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']

# Lead time classification (Short, Medium, Long)
df['lead_time_bins'] = pd.cut(df['lead_time'], bins=[0, 30, 90, np.inf], labels=['Short', 'Medium', 'Long'])

In [23]:
# Average stay duration across all bookings 
average_stay_duration = df['average_stay_duration'].mean()

# Occupancy rate per hotel (aggregate)
occupancy_rate_per_hotel = df.groupby('hotel')['is_canceled'].mean().reset_index()
occupancy_rate_per_hotel['occupancy_rate'] = 1 - occupancy_rate_per_hotel['is_canceled']
occupancy_rate_per_hotel.drop(columns=['is_canceled'], inplace=True)

# Demand per market segment (total bookings per segment)
demand_per_market_segment = df['market_segment'].value_counts().reset_index()
demand_per_market_segment.columns = ['market_segment', 'booking_count']

In [24]:
average_stay_duration

3.4279001591423066

In [25]:
occupancy_rate_per_hotel

,hotel,occupancy_rate
0,City Hotel,0.582730
1,Resort Hotel,0.722366


In [26]:
demand_per_market_segment

,market_segment,booking_count
0,Online TA,56477
1,Offline TA/TO,24219
2,Groups,19811
3,Direct,12606
4,Corporate,5295
5,Complementary,743
6,Aviation,237
7,Undefined,2


Customer and Demographics Features

In [27]:

df['guests_per_booking'] = df['adults'] + df['children'] + df['babies']

# Global metrics 
percentage_families = (df[df['guests_per_booking'] > 2].shape[0] / df.shape[0]) * 100  
percentage_repeated_guests = df['is_repeated_guest'].mean() * 100  
special_requests_avg = df['total_of_special_requests'].mean()  

In [28]:
percentage_families

12.210402881313343

In [29]:
percentage_repeated_guests

3.191222045397437

In [30]:
special_requests_avg

0.5713627607002262

Time Based Trends

In [31]:

df['waiting_list_days'] = df['days_in_waiting_list'] 

In [32]:
# Booking trend over time (total bookings per month)
booking_trend_over_time = df.groupby(['year', 'month'])['hotel'].count().reset_index()
booking_trend_over_time.columns = ['year', 'month', 'total_bookings']

# Busiest weeks (total bookings per week number)
busiest_weeks = df.groupby('arrival_date_week_number')['hotel'].count().reset_index()
busiest_weeks.columns = ['week_number', 'total_bookings']

# Effect of holiday season on bookings (total bookings during holiday vs. non-holiday)
holiday_season_effect = df.groupby('is_holiday_season')['hotel'].count().reset_index()
holiday_season_effect.columns = ['is_holiday_season', 'total_bookings']

# Average waiting list days over time 
waiting_list_trend = df.groupby(['year', 'month'])['days_in_waiting_list'].mean().reset_index()
waiting_list_trend.columns = ['year', 'month', 'average_waiting_list_days']

# Single global metric
average_waiting_list_days = df['days_in_waiting_list'].mean() 


In [33]:
booking_trend_over_time.head(6)

,year,month,total_bookings
0,2015,7,2776
1,2015,8,3889
2,2015,9,5114
3,2015,10,4957
4,2015,11,2340
5,2015,12,2920


In [34]:
busiest_weeks.head(5)

,week_number,total_bookings
0,1,1047
1,2,1218
2,3,1319
3,4,1487
4,5,1387


In [35]:
holiday_season_effect

,is_holiday_season,total_bookings
0,False,80143
1,True,39247


In [36]:
waiting_list_trend.head(4)

,year,month,average_waiting_list_days
0,2015,7,0.000000
1,2015,8,0.000000
2,2015,9,0.896558
3,2015,10,3.148880


In [37]:
average_waiting_list_days

2.321149174972778